In [12]:
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
# Import necessary libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D,MaxPool2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Model
from keras.layers import LayerNormalization
import numpy as np
!pip install tensorflow-addons
import tensorflow_addons as tfa
!pip install --quiet vit-keras
from vit_keras import vit
from keras.applications.vgg19 import preprocess_input
import pandas as pd
import glob, warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
TensorFlow Version 2.12.0


In [13]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:


# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_emotions/train',
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_emotions/test',
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
)





Found 22968 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [15]:

# Define the transformer layer
class TransformerLayer(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.mha(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

# Define the model
inputs = Input(shape=(48, 48, 1))
x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)

# Add transformer layers
embed_dim = 256
num_heads = 4
ff_dim = 512
x = tf.keras.layers.Reshape((1, -1))(x)
x = tf.keras.layers.Dense(embed_dim, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = TransformerLayer(embed_dim, num_heads, ff_dim)(x)
x = tf.keras.layers.Flatten()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(7, activation='softmax')(x)

# Create the final model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 24, 24, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 128)       73856     
                                                           

In [ ]:
# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Fine-tune the model (optional)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)],
)


In [ ]:
model_json = model.to_json()
with open("ViT_model_fromScratch.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("ViT_model_fromScratch.h5")
print("Saved model to disk")
from google.colab import files
files.download('ViT_model_fromScratch.h5')
files.download('ViT_model_fromScratch.json')

In [ ]:
hs=model.fit.history

acc = hs['accuracy']
val_acc = hs['val_accuracy']
loss =  hs['loss']
val_loss = hs['val_loss']

plt.style.use('default')

fig, (ax1) = plt.subplots(1,1, figsize= (10,5))
fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

ax1.plot(range(1, len(acc) + 1), acc)
ax1.plot(range(1, len(val_acc) + 1), val_acc)
ax1.set_title('History of Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.legend(['training', 'validation'])
